In [1]:
import pandas as pd
import numpy as np
import yfinance as yf

C:\Users\swdtk\AppData\Local\Temp\ipykernel_1364\2721483381.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [17]:
def getRateDataFrame(codeStr):
    """
    関数の説明：
        codeStrで指定したレート一覧を、ヤフーファイナンスから取得する関数
    
    引数：
        codeStr:ヤフーファイナンスから取得する通貨名を指定

    戻値：
        rateDf:OHCLが記載されたDataFrame
    """    
    #ヤフーファイナンスからレートを取得
    rateDf = yf.download(codeStr, start='2004-01-01', end='2040-12-31')

    #不要な列の削除
    del rateDf['Adj Close']
    del rateDf['Volume']

    #indexを列に変換
    rateDf.reset_index(inplace= True)

    return rateDf


def makeHendoRitu(rateDf, nRaw):
    """
    関数の説明：
        n行前の始値と、現在の始値の変動率を記載する
    
    引数：
        rateDf:Date、Open、High、Low、Closeで構成されるレートの一覧
        nRaw:どの期間の変動率を取得するか（1なら1行前）

    戻値：
        rateDf:変動率を記載したDataFrame
    """
    #n行前の始値と、現在の始値の変動率を記載
    nDaysStr = str(nRaw) + 'period'
    rateDf[nDaysStr] = rateDf['Open'].shift(nRaw)
    rateDf['変動率'] = (( rateDf['Open'] / rateDf[nDaysStr] ) -1 ) * 100
    del rateDf[nDaysStr]

    return rateDf

def makePeriodDataFrame(nRaw):
    """
    関数の説明：
        各通貨の指定した期間の変動率を１つのDataFrameにまとめる関数
    
    引数：
        nRaw:指定した期間

    戻値：
        merge_Df:各通貨の指定した期間の変動率をまとめたDataFrame
    """
    merge_Df = pd.DataFrame()

    for codeKey in currencyList:
        #レート取得
        codeValue = getRateCodeDic[codeKey]
        rateDf = getRateDataFrame(codeStr=codeValue)

        #各通貨の指定した期間のDataFrameを取得する
        rateDf = makeHendoRitu(rateDf, nRaw=nRaw)
        rateDf = rateDf[['Date', '変動率']] #必要な列のみにする
        rateDf = rateDf.rename(columns={'変動率':codeKey}) #列名を通貨名に変更

        #取得してきた各通貨の変動率をマージしていく
        if len(merge_Df) == 0:
            merge_Df = rateDf
        else:
            merge_Df = pd.merge(merge_Df, rateDf, how='left', on='Date')
    
    return merge_Df


def mergeZenjitsuhi():
    """
    関数の説明：
        各通貨の前日比を計算してマージする
    
    引数：
        なし

    戻値：
        mergeZenjitsu_Df:マージしたDataFrame
    """
    mergeZenjitsu_Df = pd.DataFrame()

    for codeKey in currencyList:
        #レート取得
        codeValue = getRateCodeDic[codeKey]
        rateDf = getRateDataFrame(codeStr=codeValue)

        #各通貨の指定した期間のDataFrameを取得する
        rateDf['翌日の始値'] = rateDf['Open'].shift(-1)
        rateDf['翌日の始値比'] = ((rateDf['翌日の始値'] / rateDf['Open']) - 1) *100
        rateDf = rateDf[['Date', '翌日の始値比']] #必要な列のみにする
        rateDf = rateDf.rename(columns={'翌日の始値比':codeKey}) #列名を通貨名に変更

        #取得してきた各通貨の変動率をマージしていく
        if len(mergeZenjitsu_Df) == 0:
            mergeZenjitsu_Df = rateDf
        else:
            mergeZenjitsu_Df = pd.merge(mergeZenjitsu_Df, rateDf, how='left', on='Date')
    
    return mergeZenjitsu_Df


def getMaxMin(df):
    """
    関数の説明：
        makePeriodDataFrameで作成したDataFrameの最大列名と最小列名を取得する
    
    引数：
        df:makePeriodDataFrameで作成したDataFrame

    戻値：
        df:最大列名と最小列名を取得したDataFrame
    """
    # 最大値を持つ列を特定
    df['MAX列'] = df.apply(lambda row: row[currencyList].idxmax(), axis=1)

    # 最小値を持つ列を特定
    df['MIN列'] = df.apply(lambda row: row[currencyList].idxmin(), axis=1)
    
    return df

def mergeMaxMinResult(x,y,z):
    """
    関数の説明：
        getMaxMinで作成した3つの変動率を作成し、それをマージする関数
    
    引数：
        x=1つ目の期間
        y=2つ目の期間
        z=3つ目の期間

    戻値：
        merge_Df:3つの結果をマージしたDataFrame
    """
    merge_Df = pd.DataFrame()

    #指定した期間の変動率をまとめる
    part1Df = makePeriodDataFrame(nRaw=x)
    #どの通貨が最大と最小かを求める
    part1Df = getMaxMin(df=part1Df)
    part1Df = part1Df[['Date','MAX列','MIN列']]

    #指定した期間の変動率をまとめる
    part2Df = makePeriodDataFrame(nRaw=y)
    #どの通貨が最大と最小かを求める
    part2Df = getMaxMin(df=part2Df)
    part2Df = part2Df[['Date','MAX列','MIN列']]

    #指定した期間の変動率をまとめる
    part3Df = makePeriodDataFrame(nRaw=z)
    #どの通貨が最大と最小かを求める
    part3Df = getMaxMin(df=part3Df)
    part3Df = part3Df[['Date','MAX列','MIN列']]


    merge_Df = pd.merge(part1Df, part2Df, how='left', on='Date')
    merge_Df = pd.merge(merge_Df, part3Df, how='left', on='Date')    
    
    return merge_Df

In [31]:
#取得する通貨
currencyList = ['USD', 'EUR', 'GBP', 'NZD', 'AUD']

#取得してくるコード一覧（すべてクロス円）
getRateCodeDic = {'USD':'USDJPY=X','EUR':'EURJPY=X', 'GBP':'GBPJPY=X', 'NZD':'NZDJPY=X', 'AUD':'AUDJPY=X'}

#各期間の最大最小をまとめる
merge_Df = mergeMaxMinResult(x=1, y=10, z=50)

#それぞれの期間が一致したらフラグを立てる
merge_Df['BuyFlg']=0
condition = (merge_Df['MAX列_x']==merge_Df['MAX列_y']) & ( merge_Df['MAX列_x']==merge_Df['MAX列'])
merge_Df.loc[condition, 'BuyFlg'] = merge_Df['MAX列_x']

merge_Df['SellFlg']=0
condition = (merge_Df['MIN列_x']==merge_Df['MIN列_y']) & ( merge_Df['MIN列_x']==merge_Df['MIN列'])
merge_Df.loc[condition, 'SellFlg'] = merge_Df['MIN列_x']

#各通貨の現在のOpneと翌日のOpneとの比較結果をまとめる
mergeZenjitsu_Df = mergeZenjitsuhi()

#merge_DfとmergeZenjitsu_Dfをマージ
result_df = pd.merge(merge_Df, mergeZenjitsu_Df, on='Date', how='left')

#結果をまとめる
result_df['結果'] = 0
for codeKey in currencyList:
    condition = (result_df['BuyFlg']==codeKey)
    result_df.loc[condition, '結果'] = result_df[codeKey]

result_df.to_csv('sample.csv', encoding='cp932', index=False)

c:\Users\swdtk\AppData\Local\Programs\Python\Python310\lib\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
[*********************100%%**********************]  1 of 1 completed
c:\Users\swdtk\AppData\Local\Programs\Python\Python310\lib\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
[*********************100%%**********************]  1 of 1 completed
c:\Users\swdtk\AppData\Local\Programs\Python\Python310\lib\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd

C:\Users\swdtk\AppData\Local\Temp\ipykernel_1364\2689083571.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[-3.97185815e-01 -8.10327554e-01  1.71316405e+00  3.76384863e-01
  7.31679474e-02 -5.13013163e-01 -1.91983243e-01  3.46428339e-01
 -3.54646545e-01 -1.81179018e-01 -4.77646158e-01 -1.95621671e-02
 -1.53157519e-01  4.88929296e-01 -6.57768239e-02  1.02576683e-01
 -5.12154814e-01 -3.72336894e-01  5.01952878e-01 -3.42214888e-01
  2.21379813e-01 -4.04972329e-01 -2.74086616e-02 -6.27125005e-02
  5.38577718e-02  5.90842234e-01 -5.15944002e-01 -5.93268040e-02
 -2.43493635e-01  1.94121979e-01 -1.44457995e-01 -7.00674787e-01
  2.54788227e-02  6.59903916e-01  2.28185450e-01 -3.25407851e-01
  3.27442691e-01 -1.80365098e-01  8.59823795e-02 -1.08238304e+00
  1.01193384e-01  1.23472088e-01 -6.43034200e-01 -9.36726962e-01
 -1.96853854e+00 -1.92915836e-01  1.30553363e+00 -4.39959377e+00
 -1.99160537e+00  4.20397

In [13]:
rateDf

,Open,High,Low,Close,Adj Close,Volume,翌日の始値,翌日の始値比
Date,,,,,,,,
2004-01-01,106.949997,107.470001,106.949997,107.300003,107.300003,0,107.320000,0.345959
2004-01-02,107.320000,107.489998,106.730003,106.910004,106.910004,0,106.930000,-0.363399
2004-01-05,106.930000,107.320000,106.010002,106.239998,106.239998,0,106.239998,-0.645284
2004-01-06,106.239998,106.389999,105.980003,106.230003,106.230003,0,106.220001,-0.018822
2004-01-07,106.220001,106.410004,105.959999,106.180000,106.180000,0,106.190002,-0.028242
...,...,...,...,...,...,...,...,...
2024-02-12,149.167007,149.475998,148.932007,149.167007,149.167007,0,149.404999,0.159547
2024-02-13,149.404999,150.753998,149.283997,149.404999,149.404999,0,150.723999,0.882835
2024-02-14,150.723999,150.774002,150.363007,150.723999,150.723999,0,150.544006,-0.119419


In [7]:
# 各日付ごとの最大値を計算
part1Df['MAX'] = part1Df[currencyList].max(axis=1)



C:\Users\swdtk\AppData\Local\Temp\ipykernel_9640\2771253278.py:2: FutureWarning: The behavior of Series.idxmax with all-NA values, or any-NA and skipna=False, is deprecated. In a future version this will raise ValueError
  part1Df['MAX列'] = part1Df.apply(lambda row: row[currencyList].idxmax(), axis=1)
C:\Users\swdtk\AppData\Local\Temp\ipykernel_9640\2771253278.py:2: FutureWarning: The behavior of Series.idxmax with all-NA values, or any-NA and skipna=False, is deprecated. In a future version this will raise ValueError
  part1Df['MAX列'] = part1Df.apply(lambda row: row[currencyList].idxmax(), axis=1)
C:\Users\swdtk\AppData\Local\Temp\ipykernel_9640\2771253278.py:2: FutureWarning: The behavior of Series.idxmax with all-NA values, or any-NA and skipna=False, is deprecated. In a future version this will raise ValueError
  part1Df['MAX列'] = part1Df.apply(lambda row: row[currencyList].idxmax(), axis=1)
C:\Users\swdtk\AppData\Local\Temp\ipykernel_9640\2771253278.py:2: FutureWarning: The behavi

In [11]:
part1Df

,Date,USD,EUR,GBP,NZD,AUD,MAX,MAX列
0,2004-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2004-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2004-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2004-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2004-01-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
5217,2024-02-09,1.039781,0.391339,0.344688,1.024122,-0.260187,1.039781,USD
5218,2024-02-12,0.693269,0.280785,0.219057,1.603473,-0.120077,1.603473,NZD
5219,2024-02-13,1.313503,0.732996,0.623588,1.198386,0.012305,1.313503,USD
5220,2024-02-14,2.276595,1.006999,1.416519,1.039295,0.086437,2.276595,USD


In [33]:
def makeSma(dataDf, tankiSma, choukiSma):
    """
    関数の説明：
        yfinanceで取得してきた株価のDataFrameを基に、
        ゴールデンクロスなら買い、デッドクロスなら決済のフラグを作成してくれる関数。
    
    Args:
        dataDf: yfinanceで取得してきた株価のDataFrame
        tankiSma: 短期移動平均線の日数
        choukiSma: 長期移動平均線の日数

    Returns:
        tradesDf: 売買フラグを付与したDataFrameを返す。
    """
    # 短期移動平均線を計算
    dataDf['短期SMA'] = dataDf['Adj Close'].rolling(window=tankiSma).mean()

    # 長期移動平均線を計算
    dataDf['長期SMA'] = dataDf['Adj Close'].rolling(window=choukiSma).mean()

    # ポジションを保持するための変数
    position = 0

    #ポジションをとった時のPrice
    numPositon = 0    

    # 取引履歴を保存するDataFrame
    tradesDf = pd.DataFrame(columns=['Date', 'Position', 'Price'])

    # 移動平均線を用いたバックテストを実施
    for i in range(len(dataDf)):
        # 短期移動平均線が長期移動平均線を上抜けした場合、買いポジションを取る
        if dataDf['短期SMA'][i] > dataDf['長期SMA'][i] and position == 0:
            position = 1
            tradesDf = tradesDf.append({'Date': dataDf.index[i], 'Position': 'Buy', 'Price': dataDf['Adj Close'][i]}, ignore_index=True)

        # 短期移動平均線が長期移動平均線を下抜けした場合、決済
        elif dataDf['短期SMA'][i] < dataDf['長期SMA'][i] and position == 1:
            position = 0
            tradesDf = tradesDf.append({'Date': dataDf.index[i], 'Position': 'Sell', 'Price': dataDf['Adj Close'][i]}, ignore_index=True)
    
    #決済金額を追記
    tradesDf['result']=0

    for i in range(len(tradesDf)):
        #Buyなら買う
        if tradesDf.iloc[i, 1] == 'Buy':
            numPositon = tradesDf.iloc[i, 2]
        #Sellなら決済
        if tradesDf.iloc[i, 1] == 'Sell':
            tradesDf.iloc[i, 3] = (tradesDf.iloc[i, 2] - numPositon) / numPositon

    return tradesDf



In [6]:
n225_sma = makeSma(dataDf=n225_train, tankiSma=5, choukiSma=25)

C:\Users\swdtk\AppData\Local\Temp\ipykernel_13084\714882119.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tradesDf = tradesDf.append({'Date': dataDf.index[i], 'Position': 'Buy', 'Price': dataDf['Adj Close'][i]}, ignore_index=True)
C:\Users\swdtk\AppData\Local\Temp\ipykernel_13084\714882119.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tradesDf = tradesDf.append({'Date': dataDf.index[i], 'Position': 'Sell', 'Price': dataDf['Adj Close'][i]}, ignore_index=True)
C:\Users\swdtk\AppData\Local\Temp\ipykernel_13084\714882119.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tradesDf = tradesDf.append({'Date': dataDf.index[i], 'Position': 'Buy', 'Price': dataDf['Adj Close'][i]}, ignore_index=True)
C:\Users\swdtk\App

          Date Position         Price    result
0   2000-02-08      Buy  19868.880859  0.000000
1   2000-03-13     Sell  19189.929688 -0.034172
2   2000-03-24      Buy  19958.080078  0.000000
3   2000-04-18     Sell  18969.519531 -0.049532
4   2000-06-08      Buy  17004.339844  0.000000
..         ...      ...           ...       ...
235 2018-10-12     Sell  22694.660156 -0.005550
236 2018-11-12      Buy  22269.880859  0.000000
237 2018-11-15     Sell  21803.619141 -0.020937
238 2018-11-29      Buy  22262.599609  0.000000
239 2018-12-07     Sell  21678.679688 -0.026229

[240 rows x 4 columns]


In [7]:
n225_sma['result'].sum()

0.5362064095993865

In [12]:
columns = ['短期SMA', '長期SMA', '結果']
resultDf = pd.DataFrame(columns=columns)

In [35]:
resultDf.to_csv('result3.csv', index=False, encoding='cp932')



In [40]:
import warnings
warnings.filterwarnings('ignore')

#短期線
for i in range(51, 250):

    #長期線    
    for j in range(i+1, 300):
        
        n225_sma = makeSma(dataDf=n225_train, tankiSma=i, choukiSma=j)

        # 結果を保存
        new_row = {'短期SMA': i, '長期SMA': j, '結果': n225_sma['result'].sum()}
        resultDf = resultDf.append(new_row, ignore_index=True)
        
resultDf.to_csv('result.csv', index=False, encoding='cp932')

In [36]:
# 日経225のデータをyfinanceから取得
n225_test = yf.download('^N225', start='2019-01-01', end='2023-12-31')

# テストデータで検証
n225_sma = makeSma(dataDf=n225_test, tankiSma=195, choukiSma=296)

[*********************100%***********************]  1 of 1 completed


In [37]:
n225_sma['result'].sum()

0.2498771142944017

In [26]:
n225_sma.to_csv('test.csv')